In [564]:
import psycopg2
from psycopg2.extensions import AsIs
import psycopg2
import pandas as pd
import numpy as np

In [565]:
%run -i ../sql_helpers.py
display(db_credentials)

{'dbname': 'gta',
 'port': 5432,
 'user': 'gta_p8',
 'password': 'r7sdkfdq',
 'host': 'ikgpgis.ethz.ch'}

In [566]:
kirchen_zuerich = pd.read_csv("csv/kirchen-zurich.csv", encoding = "utf-8", sep="	")
restaurant_zurich = pd.read_csv("csv/restaurant-zurich.csv", encoding = "utf-8", sep="	")

In [567]:
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

In [568]:
sql_string_with_placeholders = "INSERT INTO church (address_id, church_name) VALUES (%s, %s)"
cur.mogrify(sql_string_with_placeholders,(100, "kirche"))

b"INSERT INTO church (address_id, church_name) VALUES (100, 'kirche')"

In [569]:
conn.commit()
conn.close()

Kirchen

In [570]:
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

In [571]:
city_id_array = np.empty(0)

for idx, row in kirchen_zuerich.iterrows():
    #prepare input values
    church_name = row['name'] #<class 'str'> x

    address_street = row['addr:street'] #<class 'str'>
    if(type(address_street) == float):
        address_street = 'keine Adresse vorhanden!'

    address_number = row['addr:housenumber'] #<class 'str'>
    if(type(address_number) == float):
        address_number = '-'

    city_postcode = row['addr:postcode'] #<class 'str'>
    if pd.isna(city_postcode) == True:
        city_postcode = '-'
    else:
        city_postcode = int(city_postcode)
        city_postcode = str(city_postcode)

    city_name = row['addr:city'] #<class 'str'>
    if(type(city_name) == float):
        city_name = '-'

    lon = row["@lon"] #<class 'float'>

    lat = row["@lat"] #<class 'float'>

    #EXECUTE sql_commands:

    #city_table
    sql_fill_city = "INSERT INTO gta_p8.city (city_postcode, city_name) SELECT '"+city_postcode+"', '"+city_name+"' WHERE NOT EXISTS (select * from gta_p8.city where city_postcode='"+city_postcode+"');"

    cur.execute(sql_fill_city, (city_postcode, city_name))

    #address_table_preparation
    cur.execute("SELECT city_id FROM gta_p8.city WHERE city_postcode='"+city_postcode+"';")
    result = cur.fetchone()
    city_id_array = np.append(city_id_array, result[0])





In [572]:
conn.commit()
conn.close()

In [573]:
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

In [574]:
address_id_array = np.empty(0)

for idx, row in kirchen_zuerich.iterrows():
    #prepare input values
    church_name = row['name'] #<class 'str'> x

    address_street = row['addr:street'] #<class 'str'>
    if(type(address_street) == float):
        address_street = 'keine Adresse vorhanden!'

    address_number = row['addr:housenumber'] #<class 'str'>
    if(type(address_number) == float):
        address_number = '-'

    city_postcode = row['addr:postcode'] #<class 'str'>
    if pd.isna(city_postcode) == True:
        city_postcode = '-'
    else:
        city_postcode = int(city_postcode)
        city_postcode = str(city_postcode)

    city_name = row['addr:city'] #<class 'str'>
    if(type(city_name) == float):
        city_name = '-'

    lon = row["@lon"] #<class 'float'>

    lat = row["@lat"] #<class 'float'>

    #EXECUTE sql_commands:

    #address_table
    city_id_fk = str(int(city_id_array[idx]))

    sql_fill_address = "INSERT INTO gta_p8.address (address_street, address_number, city_id) SELECT '"+address_street+"', '"+address_number+"', '"+city_id_fk+"' WHERE NOT EXISTS (select * from gta_p8.address where address_street='"+address_street+"' AND address_number='"+address_number+"');"
  
    cur.execute(sql_fill_address, (address_street, address_number, city_id_fk))

    #kirchen_table_preparation
    cur.execute("SELECT address_id FROM gta_p8.address WHERE address_street='"+address_street+"' AND address_number='"+address_number+"';")
    result = cur.fetchone()
    address_id_array = np.append(city_id_array, result[0])



In [578]:
conn.commit()
conn.close()

[ 85.  86.  86.  87.  87.  88.  86.  86.  86.  86.  88.  86.  89.  86.
  90.  91.  91.  88.  90.  92.  93.  86.  94.  95.  95.  96.  97.  86.
  90.  98.  86.  88.  94.  95.  92.  94.  94.  87.  86.  87.  94.  99.
  99.  99.  85. 100.  94.  88. 101.  85.  94.  89.  96.  86.  90.  86.
  86.  86.  86.  86.  86.  93.  86. 102. 103.  86.  86.  86.  87.  86.
  88. 104.  94.  94.  90. 104.  86. 102. 102.  94.  95.  90. 102. 100.
  91.  91.  91.  91.  86.  86.  94.  96.  87. 101.  87.  94. 100. 100.
 104.  86. 105. 105.  97. 105.  85.  93.  86.  93.  86. 103.  86. 103.
  86.  86.  86.  86.  86.  86.  86.  86. 941.]


In [576]:
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

In [577]:
for idx, row in kirchen_zuerich.iterrows():
    #prepare input values
    church_name = row['name'] #<class 'str'> x

    address_street = row['addr:street'] #<class 'str'>
    if(type(address_street) == float):
        address_street = 'keine Adresse vorhanden!'

    address_number = row['addr:housenumber'] #<class 'str'>
    if(type(address_number) == float):
        address_number = '-'

    city_postcode = row['addr:postcode'] #<class 'str'>
    if pd.isna(city_postcode) == True:
        city_postcode = '-'
    else:
        city_postcode = int(city_postcode)
        city_postcode = str(city_postcode)

    city_name = row['addr:city'] #<class 'str'>
    if(type(city_name) == float):
        city_name = '-'

    lon = row["@lon"] #<class 'float'>

    lat = row["@lat"] #<class 'float'>

    #EXECUTE sql_commands:
    address_id_fk = str(int(address_id_array[idx]))

    sql_fill_address = "INSERT INTO gta_p8.church (church_name, address_id) SELECT '"+church_name+"', '"+address_id_fk+"' WHERE NOT EXISTS (select * from gta_p8.church where church_name='"+church_name+"');"

    cur.execute(sql_fill_address, (church_name, address_id_fk))



ForeignKeyViolation: insert or update on table "church" violates foreign key constraint "church_address_id_fkey"
DETAIL:  Key (address_id)=(85) is not present in table "address".


In [ ]:
conn.commit()
conn.close()

## Restaurants

In [ ]:
for idx, row in restaurant_zurich.iterrows():
    #prepare input values
    restaurant_name = row['name'] #<class 'str'>

    restaurant_website = row['website'] #<class 'str'>
    if(type(restaurant_website) == float):
        restaurant_website = 'keine Website vorhanden!'

    restaurant_type = row['cuisine']
    if(type(restaurant_type) == float):
        restaurant_type = 'keine Kategorie!'

    address_street = row['addr:street'] #<class 'str'>
    if(type(address_street) == float):
        address_street = 'keine Adresse vorhanden!'

    address_number = row['addr:housenumber'] #<class 'str'>
    if(type(address_number) == float):
        address_number = '-'

    city_postcode = row['addr:postcode'] #<class 'str'>
    if pd.isna(city_postcode) == True:
        city_postcode = '-'
    else:
        city_postcode = str(city_postcode)

    city_name = row['addr:city'] #<class 'str'>
    if(type(city_name) == float):
        city_name = '-'   

    lon = row["@lon"] #<class 'float'>

    lat = row["@lat"] #<class 'float'>
    